In [1]:
from google.colab import drive
drive.mount('/content/drive')#,force_remount=True)

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.feature_selection import f_classif
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn import metrics
from scipy import stats
import statsmodels.api as sm
import math
import re
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor
import sys
import pandas.core.algorithms as algos
pd.pandas.set_option('display.max_columns',None)
from keras.layers import Dense, concatenate,Flatten,LSTM, Embedding, Input,Dropout

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
train=pd.read_csv('/content/drive/MyDrive/Fake News Detection/FND/train.csv')

In [4]:
train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
features=[i for i  in train.columns if i not in ['label']]

In [6]:
features

['id', 'title', 'author', 'text']

In [7]:
train.shape

(20800, 5)

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [9]:
train.fillna('unavailable',inplace=True)

In [10]:
test=pd.read_csv('/content/drive/MyDrive/Fake News Detection/FND/test.csv')

In [11]:
test.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [12]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5200 entries, 0 to 5199
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      5200 non-null   int64 
 1   title   5078 non-null   object
 2   author  4697 non-null   object
 3   text    5193 non-null   object
dtypes: int64(1), object(3)
memory usage: 162.6+ KB


In [13]:
test.fillna('unavailable',inplace=True)

In [14]:
df=pd.concat([train,test],axis='rows')

In [15]:
df['comb']=df['author']+"_"+df['title']+"_"+df['text']      # Combined all the features 

In [16]:
df.sample(10)

,id,title,author,text,label,comb
12325,12325,Mulvaney: A ’Good’ Government Shutdown Would ’...,Pam Key,"Sunday on CBS’s “Face the Nation,” White House...",0.0,Pam Key_Mulvaney: A ’Good’ Government Shutdown...
4111,24911,Dragon Sighting in Thunderstorm,unavailable,www.paranormalelite.com 9 \nA supposed dragon ...,NaN,unavailable_Dragon Sighting in Thunderstorm_ww...
12823,12823,Can The American People Defeat The Oligarchy T...,pcr3,Can The American People Defeat The Oligarchy T...,1.0,pcr3_Can The American People Defeat The Oligar...
10991,10991,Former Black Panther: ’Illegitimate Congressma...,Jerome Hudson,Black conservative author and activist Mason...,0.0,Jerome Hudson_Former Black Panther: ’Illegitim...
1353,22153,What Trump Accomplished During ‘Positive and P...,Charlie Spiering,President Donald Trump discussed several issue...,NaN,Charlie Spiering_What Trump Accomplished Durin...
5079,25879,An Alabama Reader Is Pleased To Report That Je...,VDARE.com Reader,"November 7, 2016, 7:46 pm A+ | a- \nFrom: Be...",NaN,VDARE.com Reader_An Alabama Reader Is Pleased ...
16405,16405,FBI Insider: Clinton Emails Linked To Politica...,Sean Adl-Tabatabai,"Posted on October 31, 2016 by Sean Adl-Tabatab...",1.0,Sean Adl-Tabatabai_FBI Insider: Clinton Emails...
5616,5616,7 Daily Habits That Are Making You Fat,Brian,Many of the things we do on a daily basis may ...,1.0,Brian_7 Daily Habits That Are Making You Fat_M...
16858,16858,"Angelina Jolie to Divorce Brad Pitt, Ending ‘B...",Valeriya Safronova,It was a grand Hollywood romance that made its...,0.0,Valeriya Safronova_Angelina Jolie to Divorce B...
15307,15307,Could the latest solar storm warnings bring an...,Ryan Banister,VIDEOS Could the latest solar storm warnings b...,1.0,Ryan Banister_Could the latest solar storm war...


In [17]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string
import re
string.punctuation

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [18]:
wordnet=WordNetLemmatizer()
stemmer=PorterStemmer()
def clean(text):
  # text="".join([char for char in text if char not in string.punctuation])
  text="".join([re.sub('[^a-zA-Z]',' ',char) for char in text ])
  text=text.lower()
  text=text.split()
  text=[stemmer.stem(word) for word in text if word not in set(stopwords.words("english"))]
  text=" ".join(text)
  return text

In [19]:
df['comb']=df['comb'].apply(clean)

In [20]:
df.head()

,id,title,author,text,label,comb
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1.0,darrel lucu hous dem aid even see comey letter...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0.0,daniel j flynn flynn hillari clinton big woman...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1.0,consortiumnew com truth might get fire truth m...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1.0,jessica purkiss civilian kill singl us airstri...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1.0,howard portnoy iranian woman jail fiction unpu...


In [21]:
#!pip install tensorflow

In [22]:
import tensorflow
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Bidirectional

In [23]:
vocab_size=10000   # Vocabulary size

In [24]:
title=df['comb']
title[0]

0    darrel lucu hous dem aid even see comey letter...
0    david streitfeld specter trump loosen tongu pu...
Name: comb, dtype: object

In [25]:
one_hot_r=[one_hot(words, vocab_size) for words in title]

In [26]:
#one_hot_r[:1]


In [27]:
# embed 
sent_len=800
embedded_docs=pad_sequences(one_hot_r,padding='post',maxlen=sent_len)

In [28]:
embedded_docs.shape

(26000, 800)

In [29]:
sgd=tensorflow.keras.optimizers.SGD(learning_rate=0.01, decay=1e-6,momentum=0.9, nesterov=True)
rms = tensorflow.keras.optimizers.RMSprop()
nadam=tensorflow.keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Nadam")

In [30]:
vocab_size

10000

In [31]:
from keras.models import Model

In [32]:
embedding_vector_features=50
model=Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=sent_len))
model.add(LSTM(100))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer=nadam,metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 800, 50)           500000    
                                                                 
 lstm (LSTM)                 (None, 100)               60400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 560,501
Trainable params: 560,501
Non-trainable params: 0
_________________________________________________________________
None


In [33]:
len(embedded_docs)

26000

In [34]:
X=embedded_docs[:train.shape[0]]
y=df['label'][:train.shape[0]]
x_test=embedded_docs[train.shape[0]:]

In [35]:
X.shape

(20800, 800)

In [36]:
X  ##main train data
#y #main label data

array([[3997,  422, 2576, ...,    0,    0,    0],
       [8602, 2130, 4461, ...,    0,    0,    0],
       [2044, 7409, 3285, ...,    0,    0,    0],
       ...,
       [6299, 2130, 6265, ...,    0,    0,    0],
       [1627, 5772, 1920, ...,    0,    0,    0],
       [7910, 5681, 9796, ...,    0,    0,    0]], dtype=int32)

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [38]:
y_test[:5]

14649    1.0
9231     1.0
6473     0.0
18736    0.0
12347    1.0
Name: label, dtype: float64

In [ ]:
history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=15,batch_size=64)

Epoch 1/15
260/260 [==============================] - 228s 868ms/step - loss: 0.6836 - accuracy: 0.5576 - val_loss: 0.6256 - val_accuracy: 0.6731
Epoch 2/15
260/260 [==============================] - 227s 872ms/step - loss: 0.5759 - accuracy: 0.7245 - val_loss: 0.5141 - val_accuracy: 0.7740
Epoch 3/15
260/260 [==============================] - 224s 862ms/step - loss: 0.5055 - accuracy: 0.7730 - val_loss: 0.5068 - val_accuracy: 0.7726
Epoch 4/15
260/260 [==============================] - 225s 865ms/step - loss: 0.4599 - accuracy: 0.8115 - val_loss: 0.4565 - val_accuracy: 0.8257
Epoch 5/15
260/260 [==============================] - 223s 860ms/step - loss: 0.4726 - accuracy: 0.8059 - val_loss: 0.4770 - val_accuracy: 0.7933
Epoch 6/15
260/260 [==============================] - 224s 862ms/step - loss: 0.4549 - accuracy: 0.8121 - val_loss: 0.4604 - val_accuracy: 0.8149
Epoch 7/15
260/260 [==============================] - 223s 859ms/step - loss: 0.4346 - accuracy: 0.8182 - val_loss: 0.3770 -

In [ ]:
history.history??

In [ ]:
y_pred_prob=np.array(a.predict(X_train))[:,0]
y_pred_valid=np.array(a.predict(X_test))[:,0]
y_pred=np.array(a.predict(X_test))[:,0]

NameError: ignored

In [ ]:
y_predicted1 = (model.predict(X_test) > 0.5).astype("int32")

In [ ]:
y_predicted1

In [ ]:
from sklearn.metrics import precision_recall_curve
precisions, recalls, thresholds = precision_recall_curve(y_test, y_pred_valid)

In [ ]:
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
  plt.plot(thresholds, precisions[:-1], "b--", label="Precision")
  plt.plot(thresholds, recalls[:-1], "g-", label="Recall")
[...]
plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
plt.show()
plt.savefig('foo.png')

In [ ]:
def cutoff_youdens_j(fpr,tpr,thresholds):
  a = tpr-fpr
  b = sorted(zip(a,thresholds))
  return b[-1][1] 

In [ ]:
fpr, tpr, threshold = metrics.roc_curve(y_test, y_pred_valid)  ##fpr=false +ve rate and tpr = true - rate

In [ ]:
cutoff = cutoff_youdens_j(fpr, tpr, threshold)

In [ ]:
A=[]
B=[]
C=[]
for i in range(len(threshold)):
  predicted = pd.DataFrame()
  predicted["label"] = y_pred_prob
  predicted["label"] = np.where(predicted["label"] > float(threshold[i]), 1, 0)
  pred_valid = pd.DataFrame()
  pred_valid["label"] = y_pred_valid
  pred_valid["label"] = np.where(pred_valid["label"] > float(threshold[i]), 1, 0)
  A.append(metrics.accuracy_score(y_train, predicted))
  B.append(metrics.accuracy_score(y_test, pred_valid))
  C.append(threshold[i])
acc=pd.DataFrame(C,columns=['threshold'])
acc['train_acc']=A
acc['test_acc']=B
acc.sort_values(by='test_acc',ascending=False,inplace=True)

In [ ]:
B[:10]

In [ ]:
predicted = pd.DataFrame()
predicted["label"] = y_pred_prob
predicted["label"] = np.where(predicted["label"] > float(cutoff), 1, 0)
predicted

pred_valid = pd.DataFrame()
pred_valid["label"] = y_pred_valid
pred_valid["label"] = np.where(pred_valid["label"] > float(cutoff), 1, 0)
pred_valid

predictions = pd.DataFrame()
predictions['label'] = y_pred
predictions['label'] = np.where(predictions['label'] > float(cutoff), 1, 0)
predictions

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
conf_matrix = metrics.confusion_matrix(y_train, predicted)
print(conf_matrix)
acc_train = metrics.accuracy_score(y_train, predicted)
print(acc_train)
precision_train = metrics.precision_score(y_train, predicted)
print(precision_train)
sensitivity_train = metrics.recall_score(y_train, predicted)
print(sensitivity_train)
specificity_train = conf_matrix[0,0] / (conf_matrix[0,0] + conf_matrix[0,1])
print(specificity_train)
roc_auc_score(y_train, predicted)

In [ ]:
conf_matrix = metrics.confusion_matrix(y_test, pred_valid)
print(conf_matrix)

acc_train = metrics.accuracy_score(y_test, pred_valid)
print(acc_train)
precision_train = metrics.precision_score(y_test, pred_valid)
print(precision_train)
sensitivity_train = metrics.recall_score(y_test, pred_valid)
print(sensitivity_train)
specificity_train = conf_matrix[0,0] / (conf_matrix[0,0] + conf_matrix[0,1])
print(specificity_train)
roc_auc_score(y_test, pred_valid)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,pred_valid)
cm

In [ ]:
import seaborn as sn
plt.figure(figsize=(10,7))
sn.heatmap(cm,annot=True)
plt.xlabel('Predicted')
plt.ylabel('Truth')
#plt.savefig('foo.png')

In [ ]:
from sklearn.metrics import f1_score
print(f1_score(y_train, predicted,average='macro'))
f1_score(y_test, pred_valid,average='macro')

In [ ]:
submit=pd.read_csv('/content/drive/MyDrive/Fake News Detection/FND/submit.csv')

In [ ]:
submit.head()

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_predicted1)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_predicted1))

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import PrecisionRecallDisplay

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt
cm = confusion_matrix(y_test,y_predicted1)
cm
plt.figure(figsize=(10,7))
sn.heatmap(cm,annot=True)
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.savefig('confusion_matrix_LSTM.png')

In [ ]:
plt.plot(history.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train'],loc='upper left')
plt.savefig('model_accuracy')
plt.savefig('model_accuracy_vs_epoch_LSTM.png')

In [ ]:
fpr, tpr, _ = roc_curve(y_test, y_predicted1)#, pos_label=clf.classes_[1])
roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr).plot()
plt.savefig('fpr_vs_tpr_LSTM.png')

In [ ]:
prec, recall, _ = precision_recall_curve(y_test, y_predicted1)
pr_display = PrecisionRecallDisplay(precision=prec, recall=recall).plot()
plt.savefig('precision_vs_recall.png')

In [ ]:
plt.plot(history.history['loss'])
plt.title('model loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.savefig('model_loss_vs_epoch_LSTM.png')

In [ ]:
from keras.models import Sequential

In [ ]:
##BiLSTM Model
embedding_vector_features=50
model1=Sequential()
model1.add(Embedding(vocab_size,embedding_vector_features,input_length=sent_len))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.2))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer=nadam,metrics=['accuracy'])
print(model1.summary())

  # used Bidirectional LSTM

In [ ]:
history1 = model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=12,batch_size=124)

In [ ]:
history1.history??

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,pred_valid)
cm

In [ ]:
#import pickle
#with open('model.pkl','wb') as f:
 #   pickle.dump(history1,f)
    

In [ ]:
#model.save('lstmmodel.h5')

In [ ]:
news = {}

In [ ]:
news = {}
news['title'] = 'UN vote on Ukraine shows geopolitical shift in South Asia, observers say'
news['author'] = 'Anil Giri'
news['text'] = 'On Wednesday, the United Nations General Assembly passed a resolution demanding Russia immediately, completely and unconditionally withdraw all of its military forces from the territory of Ukraine within its internationally recognised borders.Nepal was among 141 member states—out of 193—to vote in favour of the UN resolution. Nepals two giant neighbours, India and China, abstained. Five—Belarus, North Korea, Eritrea and Syria and Russia naturally—voted against.South Asian nations were evenly divided. Besides Nepal, Bhutan, the Maldives and Afghanistan voted in favour of Ukraine. Afghanistans vote, however, remains asterisked because at the UN, its representatives are from the days before the Taliban takeover of the country. Besides India, Pakistan, Sri Lanka and Bangladesh abstained.'
testData = pd.DataFrame({'y':news}).transpose()
print(testData)
    #result = reloadModel.predict(testData)[0]

In [ ]:
testData.shape

In [ ]:
from keras import models
from keras.models import load_model

In [ ]:
model.save('final_model.h5')

In [ ]:
new_model = load_model('final_model.h5')

In [ ]:
new_model.summary()

In [ ]:
#y_predicted1 = (model.predict(X_test) > 0.5).astype("int32")

In [ ]:
testData['comb']=testData['author']+"_"+testData['title']+"_"+testData['text'] 
testData['comb']=testData['comb'].apply(clean)

In [ ]:
title1=testData['comb']
one_hot_r1=[one_hot(words, vocab_size) for words in title1]

In [ ]:
embedded_docs1=pad_sequences(one_hot_r1,padding='post',maxlen=sent_len)

In [ ]:
new_model.predict(embedded_docs1)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


In [ ]:
pipe = make_pipeline(StandardScaler(), LogisticRegression())
a = pipe.fit(X_train, y_train)  # apply scaling on training data
logistic_score =  pipe.score(X_test, y_test)  # # apply scaling on testing data, without leaking training data.
lr_prediction = a.predict(X_test)

In [ ]:
#logistic_model = LogisticRegression()
#logistic_model.fit(X_train, y_train)
logistic_accuracy = metrics.accuracy_score(lr_prediction,y_test)

In [ ]:
logistic_accuracy

In [ ]:
logistic_score

In [ ]:
import pickle
pickle.dump(a, open("logistic_model.pkl", "wb"))

# load the model
#model = pickle.load(open("model.pkl", "rb"))

# use model to predict
#y_pred = model.predict(X_input)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt

In [ ]:
cm_logistic = confusion_matrix(y_test,lr_prediction)
cm_logistic
plt.figure(figsize=(10,7))
sn.heatmap(cm_logistic,annot=True)
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.savefig('confusion_matrix_logistic.png')

In [ ]:
from sklearn.metrics import f1_score
print(f1_score(y_train, predicted,average='macro'))
f1_score(y_test, pred_valid,average='macro')

In [ ]:
y_pred_prob=np.array(a.predict(X_train))[:,0]
y_pred_valid=np.array(a.predict(X_test))[:,0]
y_pred=np.array(a.predict(X_test))[:,0]

In [ ]:
width = 0.25

score = [97.48, 91.00,97.74,92.25,61.65]

# Plotting the bars
fig, ax = plt.subplots(figsize=(10,5))

pos = list(range(len(score)))

plt.bar(pos, score, width, alpha=0.5, color='#EE3224', label='Development set')

# Set the y axis label
ax.set_ylabel('Accuracy score')

# Set the chart's title
ax.set_title('Accuracy Scores for all algorithms')

# Set the position of the x ticks
ax.set_xticks([p + 0.5 * width for p in pos])

# Set the labels for the x ticks
ax.set_xticklabels(['LSTM_traning','LSTM_validation', 'Bi-LSTM_traning','Bi-LSTM_validation', 'Logistic regression'])
ax.set_yticks([0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100])


plt.legend(['Algorithm'], loc='upper left')


plt.xlim(min(pos)-width, max(pos)+width*4)
plt.ylim([0, max(score)*1.3])

plt.grid()
plt.show()